In [8]:
import os
from osgeo import gdal
import numpy as np
import cv2

## Selected date

In [2]:
date_list = ['20210101',
 '20210111',
 '20210116',
 '20210126',
 '20210205',
 '20210215',
 '20210220',
 '20210225',
 '20210302',
 '20210307',
 '20210312',
 '20210327',
 '20210401',
 '20210421',
 '20210511',
 '20210521',
 '20210531',
 '20210620',
 '20210705',
 '20210804',
 '20210918',
 '20211008',
 '20211117',
 '20211122',
 '20211127',
 '20211207',
 '20211212',
 '20211217',
 '20211222',
 '20211227']

In [5]:
date_selected = date_list[12:25]
print(date_selected)

['20210401', '20210421', '20210511', '20210521', '20210531', '20210620', '20210705', '20210804', '20210918', '20211008', '20211117', '20211122', '20211127']


## Utils function

In [4]:
def read_img(date,band):
    base_dir = r"D:\Hackathon\Varuna Hackathon 2022\sentinel-2 image\2021"
    folder = 'IMG_DATA'
    img_path ='47PQS_'+date+'_'+band+'.jp2'
    read_path = os.path.join(base_dir,date,folder,img_path)
    #print(read_path)
    temp = gdal.Open(read_path)
    temp_arr = temp.GetRasterBand(1).ReadAsArray().astype('float32')
    return temp_arr
def get_features_bandstack(date):
    img3_arr=read_img(date,'B03')
    img4_arr=read_img(date,'B04')
    img8_arr=read_img(date,'B08')
    # NDVI
    ndvi = (img8_arr-img4_arr)/(img8_arr+img4_arr)
    ndvi = ndvi.reshape(2051,2051,1)
    # GNDVI
    gndvi = (img8_arr - img3_arr) / (img8_arr + img3_arr)
    gndvi = gndvi.reshape(2051,2051,1)
    # SAVI
    L = 0.428
    savi = (img8_arr - img4_arr) / (img8_arr + img4_arr + L) * (1.0 + L)
    savi = savi.reshape(2051,2051,1)
    features = np.concatenate([ndvi, gndvi, savi], axis=2)
    return features
def NDVI(date):
    img4_arr=read_img(date,'B04')
    img8_arr=read_img(date,'B08')
    ndvi = (img8_arr-img4_arr)/(img8_arr+img4_arr)
    return ndvi
def SAVI(date):
    L = 0.428
    img4_arr=read_img(date,'B04')
    img8_arr=read_img(date,'B08')
    savi = (img8_arr - img4_arr) / (img8_arr + img4_arr + L) * (1.0 + L)
    return savi

## Group selected date

In [9]:
date4 = []
for i in range(0,len(date_selected)):
    if len(date_selected[i:i+4]) == 4:
        date4.append(date_selected[i:i+4])

In [10]:
date4

[['20210401', '20210421', '20210511', '20210521'],
 ['20210421', '20210511', '20210521', '20210531'],
 ['20210511', '20210521', '20210531', '20210620'],
 ['20210521', '20210531', '20210620', '20210705'],
 ['20210531', '20210620', '20210705', '20210804'],
 ['20210620', '20210705', '20210804', '20210918'],
 ['20210705', '20210804', '20210918', '20211008'],
 ['20210804', '20210918', '20211008', '20211117'],
 ['20210918', '20211008', '20211117', '20211122'],
 ['20211008', '20211117', '20211122', '20211127']]

# Create dateaset

We are going to random crop an training image based on rasterized ground truth from shapefile.\
4 date were selected, each date contain 4 features band (NDVI, GNDVI, SAVI).\
The size of cropped image for training data is 40x40.\
Data stacking format is:\
band 1 : NDVI1 \
band 2 : GNDVI1 \
band 3 : SAVI1 \
band 4 : NDVI2 \
band 5 : GNDVI2 \
band 6 : SAVI2 \
...\
The final dimension training image is 40x40x12



## Read rasterized shapefile

In [17]:
allstage = np.load('AllCropArray.npy')
x,y = np.nonzero(allstage)
print(x,y)
ind_shuffle = np.random.permutation(len(x))
x = x[ind_shuffle]
y = y[ind_shuffle]
print(x,y)

[  28   28   28 ... 2025 2025 2025] [1780 1781 1782 ...  583  584  585]
[463 870 242 ... 128 941  45] [ 758 1546 1752 ... 1006 1179 1783]


## Random crop training data

In [18]:
allstage = np.load('AllCropArray.npy')
x,y = np.nonzero(allstage)
print(x,y)
img_sample = []
img_label = []



for date_set in date4:
    features_img1 = get_features_bandstack(date_set[0])
    features_img2 = get_features_bandstack(date_set[1])
    features_img3 = get_features_bandstack(date_set[2])
    features_img4 = get_features_bandstack(date_set[3])
    all_features = np.concatenate([features_img1, features_img2, features_img3, features_img4], axis=2)
    count = [0, 0, 0, 0]
    check = [0, 0, 0, 0]
    ind_shuffle = np.random.permutation(len(x))
    x = x[ind_shuffle]
    y = y[ind_shuffle]
    for A, B in zip(x, y):
        temp = all_features[A - 20:A + 20, B - 20:B + 20,:]
        #print(temp.shape)
        if np.all(temp > 0) and temp.shape == (40, 40,12):


            if allstage[A, B] == 1:
                count[0] += 1
                if count[0] <= 400:
                    check[0] += 1
                    img_sample.append(temp.copy())
                    img_label.append(np.array(1))

            if allstage[A, B] == 2:
                count[1] += 1
                if count[1] <= 400:
                    check[1] += 1
                    img_sample.append(temp.copy()) 
                    img_label.append(np.array(2))

            if allstage[A, B] == 3:
                count[2] += 1
                if count[2] <= 400:
                    check[2] += 1
                    img_sample.append(temp.copy()) 
                    img_label.append(np.array(3))

            if allstage[A, B] == 4:
                count[3] += 1
                if count[3] <= 400:
                    check[3] += 1
                    img_sample.append(temp.copy())
                    img_label.append(np.array(4))
            print(check)
            if sum(check) == 1600:
                break

print(count)
print(check)

[  28   28   28 ... 2025 2025 2025] [1780 1781 1782 ...  583  584  585]
[ 409  648 1901 ... 1136 1450  475] [ 766 1326 1923 ... 1772 1692 1073]


C:\Users\USER\anaconda3\envs\hackathon\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  
C:\Users\USER\anaconda3\envs\hackathon\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


[1, 0, 0, 0]
[1, 0, 1, 0]
[2, 0, 1, 0]
[2, 0, 2, 0]
[3, 0, 2, 0]
[4, 0, 2, 0]
[5, 0, 2, 0]
[6, 0, 2, 0]
[6, 0, 3, 0]
[7, 0, 3, 0]
[7, 0, 4, 0]
[8, 0, 4, 0]
[9, 0, 4, 0]
[9, 0, 5, 0]
[9, 0, 6, 0]
[9, 0, 7, 0]
[9, 0, 8, 0]
[9, 0, 9, 0]
[10, 0, 9, 0]
[10, 1, 9, 0]
[10, 1, 10, 0]
[10, 1, 11, 0]
[10, 2, 11, 0]
[10, 2, 12, 0]
[10, 2, 12, 1]
[10, 2, 13, 1]
[11, 2, 13, 1]
[11, 3, 13, 1]
[11, 3, 14, 1]
[11, 3, 15, 1]
[12, 3, 15, 1]
[12, 3, 16, 1]
[13, 3, 16, 1]
[13, 3, 17, 1]
[13, 3, 18, 1]
[13, 3, 19, 1]
[14, 3, 19, 1]
[14, 3, 20, 1]
[15, 3, 20, 1]
[15, 3, 21, 1]
[16, 3, 21, 1]
[16, 3, 21, 2]
[17, 3, 21, 2]
[18, 3, 21, 2]
[18, 3, 22, 2]
[19, 3, 22, 2]
[20, 3, 22, 2]
[21, 3, 22, 2]
[22, 3, 22, 2]
[23, 3, 22, 2]
[23, 3, 23, 2]
[24, 3, 23, 2]
[24, 3, 24, 2]
[24, 3, 25, 2]
[24, 3, 26, 2]
[24, 4, 26, 2]
[24, 4, 26, 3]
[25, 4, 26, 3]
[25, 4, 26, 4]
[26, 4, 26, 4]
[27, 4, 26, 4]
[27, 4, 26, 5]
[28, 4, 26, 5]
[29, 4, 26, 5]
[30, 4, 26, 5]
[31, 4, 26, 5]
[31, 4, 27, 5]
[31, 4, 28, 5]
[31, 4, 29, 5]
[31

[358, 30, 275, 42]
[359, 30, 275, 42]
[359, 31, 275, 42]
[359, 31, 275, 43]
[360, 31, 275, 43]
[360, 31, 276, 43]
[361, 31, 276, 43]
[362, 31, 276, 43]
[363, 31, 276, 43]
[363, 31, 277, 43]
[363, 31, 277, 44]
[363, 31, 277, 45]
[363, 31, 277, 46]
[363, 31, 278, 46]
[363, 31, 279, 46]
[363, 31, 280, 46]
[363, 32, 280, 46]
[364, 32, 280, 46]
[364, 32, 280, 47]
[365, 32, 280, 47]
[366, 32, 280, 47]
[367, 32, 280, 47]
[367, 33, 280, 47]
[368, 33, 280, 47]
[368, 33, 280, 48]
[368, 33, 281, 48]
[368, 33, 282, 48]
[368, 33, 283, 48]
[369, 33, 283, 48]
[370, 33, 283, 48]
[371, 33, 283, 48]
[371, 33, 284, 48]
[371, 33, 285, 48]
[372, 33, 285, 48]
[372, 33, 286, 48]
[373, 33, 286, 48]
[374, 33, 286, 48]
[375, 33, 286, 48]
[376, 33, 286, 48]
[377, 33, 286, 48]
[378, 33, 286, 48]
[379, 33, 286, 48]
[379, 33, 287, 48]
[380, 33, 287, 48]
[381, 33, 287, 48]
[381, 33, 288, 48]
[381, 33, 288, 49]
[381, 33, 289, 49]
[381, 33, 290, 49]
[381, 33, 290, 50]
[382, 33, 290, 50]
[382, 33, 291, 50]
[382, 34, 29

[400, 72, 400, 104]
[400, 72, 400, 104]
[400, 72, 400, 104]
[400, 73, 400, 104]
[400, 73, 400, 105]
[400, 73, 400, 105]
[400, 73, 400, 105]
[400, 73, 400, 105]
[400, 73, 400, 105]
[400, 73, 400, 105]
[400, 73, 400, 106]
[400, 73, 400, 106]
[400, 73, 400, 106]
[400, 73, 400, 106]
[400, 73, 400, 106]
[400, 73, 400, 107]
[400, 73, 400, 107]
[400, 73, 400, 107]
[400, 73, 400, 107]
[400, 73, 400, 107]
[400, 73, 400, 107]
[400, 73, 400, 107]
[400, 73, 400, 107]
[400, 73, 400, 108]
[400, 73, 400, 108]
[400, 73, 400, 108]
[400, 74, 400, 108]
[400, 74, 400, 108]
[400, 74, 400, 108]
[400, 75, 400, 108]
[400, 75, 400, 108]
[400, 75, 400, 108]
[400, 75, 400, 108]
[400, 75, 400, 108]
[400, 76, 400, 108]
[400, 76, 400, 108]
[400, 76, 400, 108]
[400, 76, 400, 108]
[400, 76, 400, 108]
[400, 76, 400, 109]
[400, 77, 400, 109]
[400, 77, 400, 109]
[400, 77, 400, 110]
[400, 77, 400, 110]
[400, 77, 400, 110]
[400, 77, 400, 110]
[400, 78, 400, 110]
[400, 78, 400, 110]
[400, 78, 400, 110]
[400, 78, 400, 110]


...

[400, 222, 400, 400]
[400, 222, 400, 400]
[400, 222, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 400, 400]
[400, 223, 40

[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 251, 400, 400]
[400, 252, 400, 400]
[400, 252, 400, 400]
[400, 252, 400, 400]
[400, 252, 400, 400]
[400, 252, 400, 400]
[400, 252, 400, 400]
[400, 253, 400, 400]
[400, 253, 400, 400]
[400, 254, 400, 400]
[400, 254, 400, 400]
[400, 254, 400, 400]
[400, 254, 400, 400]
[400, 254, 400, 400]
[400, 254, 400, 400]
[400, 254, 400, 400]
[400, 254, 400, 400]
[400, 254, 400, 400]
[400, 254, 400, 400]
[400, 254, 400, 400]
[400, 254, 400, 400]
[400, 254, 400, 400]
[400, 254, 40

[400, 300, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 400, 400]
[400, 301, 40

[400, 334, 400, 400]
[400, 334, 400, 400]
[400, 334, 400, 400]
[400, 334, 400, 400]
[400, 334, 400, 400]
[400, 335, 400, 400]
[400, 335, 400, 400]
[400, 335, 400, 400]
[400, 336, 400, 400]
[400, 336, 400, 400]
[400, 337, 400, 400]
[400, 337, 400, 400]
[400, 337, 400, 400]
[400, 338, 400, 400]
[400, 338, 400, 400]
[400, 338, 400, 400]
[400, 338, 400, 400]
[400, 338, 400, 400]
[400, 338, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 339, 400, 400]
[400, 340, 400, 400]
[400, 340, 40

[400, 375, 400, 400]
[400, 375, 400, 400]
[400, 375, 400, 400]
[400, 375, 400, 400]
[400, 375, 400, 400]
[400, 375, 400, 400]
[400, 375, 400, 400]
[400, 375, 400, 400]
[400, 375, 400, 400]
[400, 375, 400, 400]
[400, 375, 400, 400]
[400, 375, 400, 400]
[400, 375, 400, 400]
[400, 375, 400, 400]
[400, 375, 400, 400]
[400, 375, 400, 400]
[400, 375, 400, 400]
[400, 375, 400, 400]
[400, 375, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 400, 400]
[400, 376, 40

In [19]:
img_sample = np.array(img_sample)
img_label = np.array(img_label)

In [20]:
img_sample.shape

(16000, 40, 40, 12)

In [21]:
#np.savez('my_data_4day.npz',img_sample=img_sample,img_label=img_label)

In [37]:
400*4*

1600